In [1]:
import os
import logging
import src.globals as g

# Initialize logging and create missing directories
log_file = "output.txt"
fig = "fig/"

if not os.path.exists(g.fig):
    os.makedirs(g.fig)

if not os.path.exists(f"{g.fig}wake/"):
    os.makedirs(f"{g.fig}wake/")

if not os.path.exists(f"{g.fig}velocity/"):
    os.makedirs(f"{g.fig}velocity/")

logging.basicConfig(filename=log_file, filemode="w",
                    force=True, level=logging.INFO, format="%(message)s")

# Input Variables
* Wing Geometry
    * **l_**: wing span (cm) (reduce by half to be used for 2d modeling)
    * **c_**: chord length (cm) (calculated while specifying airfoil shape)
    * **n**: # of data points that define the airfoil shape.
    * **m**: # of vortex points on the airfoil
    * **camber**: Camber (not specified yet) (0 is a straight airfoil)
* Wing Motion Parameters
    * **phiT_, phiB_**: stroke angles (degrees)
    * **a_**: rotation axis offset (cm)
    * **beta_**: stroke plane angle (degrees)
    * **f_**: flapping frequency (1/sec)
    * **gMax**: max rotation (degrees)
    * **p**: rotation speed parameter (nondimentional) $p >= 4$
    * **rtOff**: rotation timing offset (nondimentional)
        * $\text{rtOff}<0$: advanced, $\text{rtOff}=0$: symmetric, $\text{rtOff}>0$: delayed
* Fluid Parameters
    * **rho_**: air density, $g/cm^3$
    * **U_, V_**: ambient velocity ($cm/sec$, assume constant)
    * **itinc**: Time increment and # of time steps option 0 (manually sepcify), 1 (automatic)
    * Velocity Contour Plot Parameters:
        * **svInc**: space-fixed velocity plot increment
        * **svMax**: space-fixed velocity plot max velocity
        * **wvInc**: wing-fixed velocity plot increment
        * **wvMax**: wing-fixed velocity plot max velocity
* Time March Variables
    * **q**: Multiplier ($0 < q <= 1$)
    * **dt**: Change in time
    * **nstep**: Total number of steps

In [2]:
import numpy as np
np.set_printoptions(precision=4)
from scipy.linalg import lu_factor, lu_solve
import src as wings

In [3]:
l_ = 0.5 * 5.0 # Change this number.
n = 101
atmp_ = 0.8
x_ = np.linspace(-atmp_, atmp_, n, endpoint=True)
camber = 0.0
y_ = camber * (atmp_ ** 2 - x_ ** 2)
c_ = x_[n - 1] - x_[0]
m = 5
phiT_ = 45
phiB_ = -45
a_ = 0
beta_ = -30
f_ = 30
gMax_ = 30
p = 5
rtOff = 0.0
rho_ = 0.001225
U_ = 100.0
V_ = 0.0
itinc = 1
svInc = 0.025
svMax = 2.5
g.svCont = np.arange(0.0, svMax + 1e-10, svInc)
wvInc = 0.1
wvMax = 7.0
g.wvCont = np.arange(0.0, wvMax + 1e-10, wvInc)
q = 1.0
dt = 0.025
nstep = 81

In [4]:
v_, t_, d_, e, c, x, y, a, beta, gMax, U, V = wings.in_data(l_, phiT_, phiB_, c_, x_, y_, a_, beta_, f_, gMax_, U_, V_)

g.delta = 0.5 * c / (m - 1) * q

if itinc == 1:
    nperiod = 1
    dt = min(c / (m - 1), 0.1 * (4 / p))
    nstep = int(nperiod * np.ceil(2/dt))

air = np.sqrt(U_ ** 2 + V_ ** 2)
fk = 2 * f_ * d_ / air
r = 0.25 * (c_ / d_) * (p / t_) * (gMax / f_)
k = fk * r

if air <= 1e-03:
    r = 0.25 * (c_ / d_) * (p / t_) * (gMax / f_)

xv, yv, xc, yc, dfc, m  = wings.mesh_r(c, x, y, n, m)

# Initialization Of Time March

* Initialize the wake vortex. (**GAMAw**)
    * GAMAw[0:2] step 1, GAMAw[2:4] step 2, ...
* Initialize teh free vortex magnitude array. (**GAMAf**)
    * This is the vortext to be shed or convected.
* Initialize the total wake vortex sum (**sGAMAw**)
* Initialize the total wake vortex number (**iGAMAw**)
* Initialize the # of vortices to be convected or shed. (**iGAMAf**)
* Initialize the free & wake vortex location array (before convection)(**ZF**)
    * ZF[0:2] step 1, ZF[2:4] step 2, ZF[4:6] step 3, ...
    * Leading edge: odd components
    * Trailing edge: even components
* Initialize the free & wake vortex location array (after convection)(**ZW**)
    * ZW[0:2] step 1, ZW[2:4] step 2, ZW[4:6] step 3, ...
    * Leading edge: odd components
    * Trailing edge: even components
* This is further transformed into a new body-fixed coordinate system.
* Linear and angular impulse arrays (**impulseLb, impulseAb, impulseLw, impulseLb**)

In [5]:
GAMAw = np.zeros(2 * nstep)
GAMAf = np.zeros(2 * nstep)
sGAMAw = 0.0
iGAMAw = 0
iGAMAf = 0
ZF = np.zeros(2 * nstep, dtype=complex)
ZW = np.zeros(2 * nstep, dtype=complex)
impulseLb = np.zeros(nstep, dtype=complex)
impulseLw = np.zeros(nstep, dtype=complex)
impulseAb = np.zeros(nstep)
impulseAw = np.zeros(nstep)
LDOT = np.zeros(nstep)
HDOT = np.zeros(nstep)

* Setup the matrix for the nonpenetration condition
* Use the wing-fixed coordinate system to calcuate the matrix coefficients.
* The matrix coefficients in the global system are identical to these and remain constant throughout the time steps.

In [6]:
MVN = wings.matrix_coef(xv, yv, xc, yc, dfc, m)
MVN_lu = lu_factor(MVN)

ZETA = 0
if g.vfplot == 1:
    if camber == 0.0:
        ZETA = wings.c_mesh(c_, d_)
    else:
        ZETA = wings.camber_mesh(c_, d_, camber)

# Time Marching

* Perform the time march using the values defined above for calculation.

In [ ]:
iterations = {
    'ZC': [],
    'NC': [],
    't': [],
    'VN': [],
    'iGAMAw': [],
    'ZV': [],
    'ZW': [],
    'GAMA': [],
    'GAMAw': [],
    'U': [],
    'V': [],
    'alp': [],
    'l': [],
    'h': [],
    'dalp': [],
    'dl': [],
    'dh': []
}

In [7]:
for istep in range(nstep):
    t = istep * dt
    alp, l, h, dalp, dl, dh = wings.airfoil_m(t, e, beta, gMax, p, rtOff, U, V)

    LDOT[istep] = dl
    HDOT[istep] = dh

    NC, ZV, ZC, ZVt, ZCt, ZWt = wings.wing_global(istep, t, a, alp, l, h, xv, yv, xc, yc, dfc, ZW, U, V)

    VN = wings.airfoil_v(ZC, ZCt, NC, t, dl, dh, dalp)
    VNW = wings.velocity_w2(m, ZC, NC, ZF, GAMAw, iGAMAw)
    
    GAMA = VN - VNW
    GAMA = np.append(GAMA, -sGAMAw)
    GAMA = lu_solve(MVN_lu, GAMA)

    impulseLb[istep] = -1j * np.sum(GAMA * ZVt)
    impulseAb[istep] = 0.5 * np.sum(GAMA * np.abs(ZVt) ** 2)
    impulseLw[istep] = -1j * np.sum(GAMAw[0:iGAMAw] * ZWt[0:iGAMAw])
    impulseAw[istep] = 0.5 * np.sum(GAMAw[0:iGAMAw] * np.abs(ZWt[0:iGAMAw]) ** 2)

    iGAMAf = 2 * (istep + 1)

    ZF[iGAMAf - 2] = ZV[0]
    ZF[iGAMAf - 1] = ZV[m - 1]

    VELF = wings.velocity_improved(ZF, iGAMAf, GAMA, m, ZV, GAMAw, iGAMAw)
    
    ZW[0:iGAMAf] = ZF[0:iGAMAf] + VELF * dt

    iGAMAw = iGAMAw + 2
    GAMAw[iGAMAf - 2] = GAMA[0]
    GAMAw[iGAMAf - 1] = GAMA[m - 1]
    sGAMAw = sGAMAw + GAMA[0] + GAMA[m - 1]

    ZF = ZW